In [98]:
import pandas as pd
import numpy as np

In [99]:
df = pd.read_csv("sp500_testdata_2.csv")

In [3]:
# df.tail()
# df.round({'N': 2, 'NPP': 3, 'FL': 2, 'SL': 2, 'Sep': 2, 'FE_FL': 2, 'AE_FL': 2})

In [100]:
# 28 columns: date,o, h, l, c, TR, N, NPP, FL, SL, Sep, TrendDir, TrendDay, FE_FL, AE_FL, Counter1, Counter2, Position, SP, EnP1_init
#             EnP2_RE, EnP3_AP, ExP1_initTP, ExP2_AP, ExP3_SL, ExP4_SL, Exp5_TrendChange, P_L
df["TR"] = ""
df.TR.loc[0] = df.high.loc[0] - df.low.loc[0]    # Initial value for TR
df.TR.loc[1:] = ""
df["N"] = ""
df["N"].loc[0] = df.high.loc[0] - df.low.loc[0]   # Initial value for N
df["N"].loc[1:] = ""
df["NPP"] = ""
df.NPP.loc[0] = 0.00     # Initial value for NPP
df["FL"] = ""
df["SL"] = ""
df.FL.loc[0] = 0.00      # Initial value for FL
df.SL.loc[0] = 0.00      # Initial value for SL
df["Sep"] = ""           # Percentage separation between FL and SL
df.Sep.loc[0] = 0.00
df["TrendDir"] = ""
df["TrendDir"] = df["TrendDir"].astype('str')
df["TrendDay"] = ""
df["FE_FL"] = ""
# df.FE_FL.round(3)       #
df["AE_FL"] = ""
df["Counter1"] = ""
df["Counter2"] = ""
df["Position"] = ""
df["SP"] = ""
df["EnP1_init"] = ""
df["EnP2_RE"] = ""
df["EnP3_AP"] = ""
df["ExP1_initTP"] = ""
df["RE_CAN"] = ""
df["ExP2_AP_TP"] = ""
df["ExP3_SL"] = ""
df["ExP4_SL"] = ""
df["ExP5_TrendChange"] = ""
df["P_L"] = ""

df[["TR", "N", "NPP", "FL", "SL", "Sep", "FE_FL", "AE_FL", "SP", "EnP1_init", "EnP2_RE", "EnP3_AP", "ExP1_initTP", "ExP2_AP_TP", "ExP3_SL", "ExP4_SL", "ExP5_TrendChange"]] = df[["TR", "N", "NPP", "FL", "SL", "Sep", "FE_FL", "AE_FL", "SP", "EnP1_init", "EnP2_RE", "EnP3_AP", "ExP1_initTP", "ExP2_AP_TP", "ExP3_SL", "ExP4_SL", "ExP5_TrendChange"]].apply(pd.to_numeric)

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [101]:
df.dtypes

Date                 object
open                float64
high                float64
low                 float64
close               float64
TR                  float64
N                   float64
NPP                 float64
FL                  float64
SL                  float64
Sep                 float64
TrendDir             object
TrendDay             object
FE_FL               float64
AE_FL               float64
Counter1             object
Counter2             object
Position             object
SP                  float64
EnP1_init           float64
EnP2_RE             float64
EnP3_AP             float64
ExP1_initTP         float64
RE_CAN               object
ExP2_AP_TP          float64
ExP3_SL             float64
ExP4_SL             float64
ExP5_TrendChange    float64
P_L                  object
dtype: object

In [102]:
df.FE_FL.dtype
# df.round({'TR': 2, 'N': 2, 'NPP': 2, 'FL': 2, 'SL': 2, 'Sep': 2, 'FE_FL': 2, 'AE_FL': 2})
# df.iloc[0:50, 0:17]

dtype('float64')

In [103]:
#  3 'for' loops: 
#   a) calculate True Range for every row after the first row
#   b) calculate N for every row after the first row
#   c) calculate NPP for every row after first row


# a)
#  True Range = max( absolute value(H - L), absolute value (H - C(previous)), absoulute value (C(previous) - L))
rownum = 1
for rownum in range(1,1500):
    df.TR.loc[rownum:] = max(abs(df.high.loc[rownum] - df.low.loc[rownum]), abs(df.high.loc[rownum] - df.close.loc[rownum-1]), abs(df.close.loc[rownum-1] - df.low.loc[rownum]))
    rownum += 1
    
#  b)
#  N = N(previous) + (TR-N(previous)) / (0.5 * 20 + 1)
Constant = 20
rownum = 1
for rownum in range(1, 1500):
    df.N.loc[rownum:] = df.N.loc[rownum - 1]+(df.TR.loc[rownum]-df.N.loc[rownum - 1])/(0.5 * (Constant + 1))
    rownum +=1
    
# c)
df.NPP.loc[rownum:].round(3)
rownum = 1
for rownum in range(1, 1500):
    df.NPP.loc[rownum:] = df.N.loc[rownum]/df.close.loc[rownum-1] * 100
    rownum += 1
    
# try replacing this with a List Comprehension    
# [df.NPP.loc[rownum:] for df.NPP.loc[rownum] in range(1.599) if df.NPP.loc[rownum] is float]



C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [104]:
# 3 min 42 sec to finish execution on 1500 rows
# 'for' loop that yields FL values after index position zero

rownum = 1
FL_period = 7
for rownum in range(1, 1500):
    df.FL.loc[rownum:] = df.FL.loc[rownum-1] + (df.close.loc[rownum] - df.FL.loc[rownum-1])/((FL_period+1)/2)
    rownum += 1
    
# 'for' loop that yields SL values after index position zero

rownum = 1
SL_period = 20
for rownum in range(1, 1500):
    df.SL.loc[rownum:] = df.SL.loc[rownum-1] + (df.close.loc[rownum] - df.SL.loc[rownum-1])/((SL_period+1)/2)
    rownum += 1    

# 'for' loop that yields Separation between FL and SL   
    
rownum = 1
for rownum in range(1, 1500):
    df.Sep.loc[rownum] = (df.FL.loc[rownum] - df.SL.loc[rownum])/df.SL.loc[rownum] * 100
    rownum += 1
    
    
# Determining TrendDir based on boolean result for question: FL > SL ?
# start at index = 25 to give ramp-up time for lags
rownum = 100
for rownum in range(25, 1500):
    if df.FL.loc[rownum] > df.SL.loc[rownum]:
        df.TrendDir.loc[rownum] = "Up"
    else:    
        df.TrendDir.loc[rownum] = "Down"
rownum += 1

# iterate through rows starting at row 101 to find where trends initiate (TrendDay=0) and continue (TrendDay>0)

rownum = 101           # start at index = 101 (TrendDay at index=100 is set to 0)
for rownum in range(25, 1500):
    if df.TrendDir.loc[rownum] == df.TrendDir.loc[rownum-1]:
        df.TrendDay.loc[rownum] = (df.TrendDay.loc[rownum-1] + 1)
    else:
        df.TrendDay.loc[rownum] = 0
rownum += 1




C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [105]:
# 1 min 23 sec to finish execution on 1500 rows

rownum = 25   # start after series for TrendDay transitions from blank (string value) to int
for rownum in range(25, 1500):
    if df.TrendDay.loc[rownum] == 0:        
        df.FE_FL.loc[rownum] = ""
        df.AE_FL.loc[rownum] = ""
    else:
        if df.TrendDir.loc[rownum] == "Up":
        
            #Favorable Excursion from FL in terms of N for long position:
            if df.high.loc[rownum] < df.FL.loc[rownum-1]:
                df.FE_FL.loc[rownum] = 0
            else:
                df.FE_FL.loc[rownum] = (df.high.loc[rownum] - df.FL.loc[rownum-1])/df.N.loc[rownum-1]

            #Adverse Excursion from FL in terms of N for for long position (display positive value):
            if df.low.loc[rownum] > df.FL.loc[rownum-1]:
                df.AE_FL.loc[rownum] = 0
            else:
                df.AE_FL.loc[rownum] = abs((df.low.loc[rownum] - df.FL.loc[rownum-1])/df.N.loc[rownum-1])
    
        elif df.TrendDir.loc[rownum] == "Down":
        
            #Favorable Excursion from FL in terms of N for Short position:
            if df.low.loc[rownum] > df.FL.loc[rownum-1]:
                df.FE_FL.loc[rownum] = 0
            else:
                df.FE_FL.loc[rownum] = (df.FL.loc[rownum-1] - df.low.loc[rownum])/df.N.loc[rownum-1]

            #Adverse Excursion from FL in terms of N for for Short position (display positive value):
            if df.high.loc[rownum] < df.FL.loc[rownum-1]:
                df.AE_FL.loc[rownum] = 0
            else:
                df.AE_FL.loc[rownum] = abs((df.FL.loc[rownum-1] - df.high.loc[rownum])/df.N.loc[rownum-1])        
            
# incrememnt to next bar
rownum += 1

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [106]:
df[["FE_FL", "AE_FL", "TrendDay", "Counter1", "Counter2", "Position", "SP", "EnP1_init", "EnP3_AP", "ExP1_initTP", "ExP2_AP_TP", "RE_CAN", "ExP3_SL", "ExP4_SL", "ExP5_TrendChange", "P_L"]] = df[["FE_FL", "AE_FL", "TrendDay", "Counter1", "Counter2", "Position", "SP", "EnP1_init", "EnP3_AP", "ExP1_initTP", "ExP2_AP_TP", "RE_CAN", "ExP3_SL", "ExP4_SL", "ExP5_TrendChange", "P_L"]].apply(pd.to_numeric)
# pd.to_numeric(df["TrendDay", "Counter1", "Counter2"], downcast='integer'))

In [121]:
df.iloc[400:460, 0:20].round({'N': 2, 'NPP': 3, 'FL': 2, 'SL': 2, 'Sep': 2, 'FE_FL': 3, 'AE_FL': 3})

,Date,open,high,low,close,TR,N,NPP,FL,SL,Sep,TrendDir,TrendDay,FE_FL,AE_FL,Counter1,Counter2,Position,SP,EnP1_init
400,6/5/2009,747.75,764.25,740.00,747.25,24.25,22.27,2.980,737.23,718.12,2.66,Up,56.0,1.376,0.000,NaN,NaN,NaN,NaN,NaN
401,6/8/2009,746.25,753.25,732.25,745.50,21.00,22.15,2.964,739.30,720.73,2.58,Up,57.0,0.719,0.224,NaN,NaN,NaN,NaN,NaN
402,6/9/2009,745.50,753.75,741.00,746.25,12.75,21.25,2.851,741.04,723.16,2.47,Up,58.0,0.652,0.000,NaN,NaN,NaN,NaN,NaN
403,6/10/2009,746.00,761.75,733.75,747.25,28.00,21.90,2.934,742.59,725.46,2.36,Up,59.0,0.975,0.343,NaN,NaN,NaN,NaN,NaN
404,6/11/2009,746.50,763.25,743.50,748.75,19.75,21.69,2.903,744.13,727.67,2.26,Up,60.0,0.944,0.000,NaN,NaN,NaN,NaN,NaN
405,6/12/2009,747.75,753.50,742.00,751.50,11.50,20.72,2.767,745.97,729.94,2.20,Up,61.0,0.432,0.098,NaN,NaN,NaN,NaN,NaN
406,6/15/2009,749.25,749.50,726.00,730.25,25.50,21.18,2.818,742.04,729.97,1.65,Up,62.0,0.170,0.964,NaN,NaN,NaN,NaN,NaN
407,6/16/2009,730.50,735.50,717.25,718.50,18.25,20.90,2.862,736.16,728.88,1.00,Up,63.0,0.000,1.171,NaN,NaN,NaN,NaN,NaN
408,6/17/2009,718.75,725.00,710.00,716.00,15.00,20.34,2.830,731.12,727.65,0.48,Up,64.0,0.000,1.252,NaN,NaN,NaN,NaN,NaN
409,6/18/2009,716.25,728.50,713.75,724.00,14.75,19.80,2.766,729.34,727.31,0.28,Up,65.0,0.000,0.854,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.dtypes

In [99]:
# df["TrendDir"] = df["TrendDir"].astype(str)
# df["TrendDay"] = df["TrendDay"].astype(int)

In [59]:
#tick = 0.25
#print(tick)
#tick_val = 12.50
#print(tick_val)
#rownum = 5                                             
#df.Position.loc[rownum] = 0
# df['Position'] = Position                               # create a column (series) and set each column slice equal to 'position' variable

In [108]:
X = 0.25   # General Risk Mgmt. multiplier

addpos = {"L" : {"1" : 0.011, "2" : 0.015, "3" : 0.0195},                            # syntax to call, enter script as follows:
          "S" : {"1" : 0.011, "2" : 0.015, "3" : 0.0195}                             # AP["L"]["2"]  yields: 0.015
          }                                                                        # SL["S"]["3"]  yields: 0.0195

stoploss = {"L" : {"1" : 0.011, "2" : 0.015, "3" : 0.0195},
            "S" : {"1" : 0.011, "2" : 0.015, "3" : 0.0195}
            }

takeprof = {"L" : {"1" : 0.011, "2" : 0.015, "3" : 0.0195},
            "S" : {"1" : 0.011, "2" : 0.015, "3" : 0.0195}
            }

re_entry = {"L" : {"1" : 0.011, "2" : 0.015, "3" : 0.0195},
            "S" : {"1" : 0.011, "2" : 0.015, "3" : 0.0195}
            }

In [109]:
#  daily Volatility Impact functions (Risk Management)

def AP():                                                     # Add Position:  to call:  AP(df.NPP.loc[rownum])
    if(df.TrendDir.loc[rownum - 1] == "Up"):
        if df.NPP.loc[rownum - 1] <= addpos["L"]["1"]:
            return df.FL.loc[rownum - 1] + 2 * X / df.N.loc[rownum - 1]     # Feel some heat to add position when price exceeds half N (2 *0.25N) below FL
        elif df.NPP.loc[rownum - 1] <= addpos["L"]["2"]:
            return df.FL.loc[rownum - 1]                                    # Feel some heat to add position when price exceeds half N (2 *0.25N) below FL
        elif df.NPP.loc[rownum - 1] <= addpos["L"]["3"]:
            return df.FL.loc[rownum - 1] - 2 * X * df.N.loc[rownum - 1]     # Feel some heat to add position when price exceeds half N (2 *0.25N) below FL
        else:
            return df.FL.loc[rownum - 1] - 2.25 * X * df.N.loc[rownum - 1]     # Feel some heat to add position when price exceeds half N (2 *0.25N) below FL
    elif (df.TrendDir.loc[rownum - 1] == "Down"):
        if df.NPP.loc[rownum - 1] < AP["S"]["1"]:
            return df.FL.loc[rownum - 1] + 2 * X * df.N.loc[rownum - 1]     # Feel some heat to add position when price exceeds half N (2 *0.25N) above FL
        elif df.NPP.loc[rownum - 1] >= AP["S"]["1"]  and  df.NPP.loc[rownum - 1] < AP["S"]["2"]:
            return df.FL.loc[rownum - 1] + 2 * X * df.N.loc[rownum - 1]     # Feel some heat to add position when price exceeds half N (2 *0.25N) above FL
        elif df.NPP.loc[rownum - 1] >= AP["S"]["2"]  and  df.NPP.loc[rownum - 1] < AP["S"]["3"]:
            return df.FL.loc[rownum - 1] + 2 * X * df.N.loc[rownum - 1]      # Feel some heat to add position when price exceeds half N (2 *0.25N) above FL
        elif df.NPP.loc[rownum - 1] >=  AP["S"]["3"]  and  df.NPP.loc[rownum - 1] <  AP["S"]["4"]:
            return df.FL.loc[rownum - 1] + 2 * X * df.N.loc[rownum - 1]       # Feel some heat to add position when price exceeds half N (2 *0.25N) above FL
        else:
            return df.FL.loc[rownum - 1] + 2 * X * df.N.loc[rownum - 1]      # Feel some heat to add position when price exceeds half N (2 *0.25N) above FL


def SL():                        # Stop Loss - must space SL multipliers proportional to AP multipliers   to call:  df.NPP.loc[rownum - 1]
    if(df.TrendDir.loc[rownum - 1] == "Up"):
        if df.NPP.loc[rownum - 1] <= stoploss["L"]["1"]:
            return df.FL.loc[rownum - 1] - X * df.N.loc[rownum - 1]     
        elif df.NPP.loc[rownum - 1] <= stoploss["L"]["2"]:
            return df.FL.loc[rownum - 1] - 3 * X * df.N.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= stoploss["L"]["3"]:
            return df.FL.loc[rownum - 1] - 6 * X * df.N.loc[rownum - 1]     
        else:
            return df.FL.loc[rownum - 1] - 8 * X * df.N.loc[rownum - 1]     #add position when price exceeds half 8*X*N (8 * 0.25N = 2N) below FL
    if(df.TrendDir.loc[rownum - 1] == "Down"):
        if df.NPP.loc[rownum - 1] < SL["S"]["1"]:
            return df.FL.loc[rownum - 1] + 4 * X * df.N.loc[rownum - 1]           # for NPP < 1.05%: too much heat when high exceeds 1N (4 * .25N) above previous FL
        elif df.NPP.loc[rownum - 1] >= SL["S"]["1"]  and  df.NPP.loc[rownum - 1] < SL["L"]["2"]:
            return df.FL.loc[rownum - 1] + 4 * X * df.N.loc[rownum - 1]           # for NPP < 1.30%: too much heat when high exceeds 1N (4 * .25N) above previous FL
        elif df.NPP.loc[rownum - 1] >= SL["S"]["2"]  and  df.NPP.loc[rownum - 1] < SL["L"]["3"]:
            return df.FL.loc[rownum - 1] + 4 * X * df.N.loc[rownum - 1]           # for NPP < 1.65%: too much heat when high exceeds 1N (4 * .25N) above previous FL
        elif df.NPP.loc[rownum - 1] >=  SL["S"]["3"]  and  df.NPP.loc[rownum - 1] <  SL["S"]["4"]:
            return df.FL.loc[rownum - 1] + 4 * X * df.N.loc[rownum - 1]           # for NPP < 1.95%: too much heat when price exceeds 1N (4 * .25N) above previous FL
        else:
            return df.FL.loc[rownum - 1] + 4 * X * df.N.loc[rownum - 1]           # for NPP > 1.95%: too much heat when price exceeds 1N (4 * .25N) above previous FL


def TP():                                                # Take Profit - to call: df.NPP.loc[rownum - 1]
    if(df.TrendDir.loc[rownum - 1] == "Up"):
        if df.NPP.loc[rownum - 1] <= takeprof["L"]["1"]:
            return df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]           # for NPP < 1.10%: take profit when price exceeds 1.25N (5*0.25N) above previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["L"]["2"]:
            return df.FL.loc[rownum - 1] + 6 * X * df.N.loc[rownum - 1]           # for NPP < 1.30%: take profit when price exceeds 1.25N (5*0.25N) above previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["L"]["3"]:
            return df.FL.loc[rownum - 1] + 7 * X * df.N.loc[rownum - 1]            # for NPP < 1.65%: take profit when price exceeds 1N (4*0.25N) above previous FL
        else:
            return df.FL.loc[rownum - 1] + 7.5 * X *df.N.loc[rownum - 1]             # for NPP > 1.95%: take profit when price exceeds 1N (4*0.25N) above previous FL
    elif (df.TrendDir.loc[rownum - 1] == "Down"):
        if df.NPP.loc[rownum - 1] < TP["S"]["1"]:
            return df.FL.loc[rownum - 1] - 6 * X * df.N.loc[rownum - 1]            # for NPP < 1.05%: take profit when price exceeds 1.5N (6*0.25N) below previous FL
        elif df.NPP.loc[rownum - 1] >= TP["S"]["1"]  and  df.NPP.loc[rownum - 1] < TP["S"]["2"]:
            return df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]             # for NPP < 1.30%: take profit when price exceeds 1.25N (5*0.25N) below previous FL
        elif df.NPP.loc[rownum - 1] >= TP["S"]["2"]  and  df.NPP.loc[rownum - 1] < TP["S"]["3"]:
            return df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]           # for NPP < 1.65%: take profit when price exceeds 1.25N (5*0.25N) below previous FL
        elif df.NPP.loc[rownum - 1] >=  TP["S"]["3"]  and  df.NPP.loc[rownum - 1] <  TP["S"]["4"]:
            return df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]           # for NPP < 1.95%: take profit when price exceeds 1.25N (5*0.25N) below previous FL
        else:
            return df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]            # for NPP > 1.95%: take profit when price exceeds 1.25N (5*0.25N) below previous FL


def RE():                                                    # Re-enter trend after AP, SL, or TP     -  to call: df.NPP.loc[rownum - 1]
    if(df.TrendDir.loc[rownum - 1] == "Up"):
        if df.NPP.loc[rownum - 1] <= re_entry["L"]["1"]:
            return df.FL.loc[rownum - 1] + 2 * X / df.N.loc[rownum - 1]     # Feel some heat to add position when price exceeds half N (2 *0.25N) below FL
        elif df.NPP.loc[rownum - 1] <= re_entry["L"]["2"]:
            return df.FL.loc[rownum - 1]                                    # Feel some heat to add position when price exceeds half N (2 *0.25N) below FL
        elif df.NPP.loc[rownum - 1] <= re_entry["L"]["3"]:
            return df.FL.loc[rownum - 1] - 2 * X * df.N.loc[rownum - 1]     # Feel some heat to add position when price exceeds half N (2 *0.25N) below FL
        else:
            return df.FL.loc[rownum - 1] - 2.25 * X * df.N.loc[rownum - 1]     # Feel some heat to add position when price exceeds half N (2 *0.25N) below FL
    elif (df.TrendDir.loc[rownum - 1] == "Down"):
        if df.NPP.loc[rownum - 1] < RE["S"]["1"]:
            return df.FL.loc[rownum - 1] - multiplier[x1] * df.N.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] >= RE["S"]["1"]  and  df.NPP.loc[rownum - 1] < RE["S"]["2"]:
            return df.FL.loc[rownum - 1] - multiplier[x2] * df.N.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] >= RE["S"]["2"]  and  df.NPP.loc[rownum - 1] < RE["S"]["3"]:
            return df.FL.loc[rownum - 1] - multiplier[x3] * df.N.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] >=  RE["S"]["3"]  and  df.NPP.loc[rownum - 1] <  RE["S"]["4"]:
            return df.FL.loc[rownum - 1] - multiplier[x4] * df.N.loc[rownum - 1]
        else:
            return df.FL.loc[rownum - 1] - multiplier[x4] *df.N.loc[rownum - 1]





In [120]:
tick = 0.25
tick_val = 12.50           
rownum = 50                                                 # start looking for uptrend starts from row 50
df.Position.loc[rownum] = 0

for rownum in range(1, 1500):
    if df.TrendDay.loc[rownum] == 0 and df.TrendDir.loc[rownum] == "Up":
        SP = df.high.loc[rownum]                            # assign the 'high' slice to the SP variable
        df.SP.loc[rownum] = SP                              # equate the SP slice with the SP variable
        rownum += 1
        while df.TrendDay.loc[rownum] > 0:              # this 'while' loop finds initial entry, records initial P_L, then advances to next day
            if df.high.loc[rownum] > SP:
                EnP1_init = SP + tick
                df.EnP1_init.loc[rownum] = EnP1_init
                df.Counter1.loc[rownum] = 1                  # initialize Counter1
                df.Position.loc[rownum] = 1                  # 'position' variable
                # df.Position.loc[rownum] = Position         # this [rownum] dataframe slice equal to this [rownum] value for position variable
                df.P_L.loc[rownum] = df.close.loc[rownum] - EnP1_init        # yes, you can add a value contained in dataframe slice with a local variable that's a float!
                rownum += 1
                break
            else:
                SP = df.high.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1
        
        while df.TrendDay.loc[rownum] > 0:
            a = AP()
            b = SL()
            c = TP()
    
            # case 1: price retraces - look to add to initial position
            if df.Position.loc[rownum] == 1 and df.low.loc[rownum] < a:
                EnP3_AP = a - tick
                df.EnP3.loc[rownum] = EnP3_AP
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Counter2.loc[rownum] = 1
                df.Position.loc[rownum] = 2
                df.P_L.loc[rownum] = (df.close.loc[rownum] - EnP1_init) + (df.close.loc[rownum] - EnP3_AP)
                rownum += 1
                continue
        
            # case 2:  take profit on initial position   
            elif df.Position.loc[rownum] == 1 and df.high.loc[rownum] > c:
                ExP1_initTP = c + tick
                df.ExP1_initTP.loc[rownum] = ExP1_initTP
                df.Counter1.loc[rownum] = 0
                df.Position.loc[rownum] = 0
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                df.P_L.loc[rownum] = ExP1_initTP - EnP1_init
                rownum += 1
                continue
        
            # case 3:  take profit on two positions
            elif df.Position.loc[rownum] == 2 and df.high.loc[rownum] > c:
                ExP1_initTP = c + tick
                ExP2_AP_TP = c + tick
                df.Counter1.loc[rownum] = 0
                df.Counter2.loc[rownum] = 0
                df.Position.loc[rownum] = 0
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                df.P_L.loc[rownum] = (ExP1_initTP - EnP1_init) + (ExP2_AP-TP - EnP3_AP)
                rownum += 1
                continue
        
             # case 4:  RM functions do not impact initial position so carry to next day
            elif df.Position.loc[rownum] == 1 and df.high.loc[rownum] <=c and df.low.loc[rownum] >= a:
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Position.loc[rownum] = 1
                df.P_L.loc[rownum] = df.close.loc[rownum] - EnP1_init
                rownum += 1
                continue
    
            # case 5: RM functions do no impact position with TWO contracts so carry to next day
            elif df.Position.loc[rownum] == 2 and df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Counter2.loc[rownum] = df.Counter2.loc[rownum-1] + 1
                df.Position.loc[rownum] = 2
                df.P_L.loc[rownum] = (df.close.loc[rownum] - EnP1_init) + (df.close.loc[rownum] - EnP3_AP)
                rownum += 1
                continue
        
            # case 6:  zero positions due to TP trigger and looking for Re-entry on TWO positions
            elif df.Position.loc[rownum] == 0 and df.RE_CAN.loc[rownum] == True:
                d = RE(df.NPP.loc[rownum-1])
                if low <= d:
                    EnP1_init = d                         # don't use unique variable for re-entering:  EnP2_RE = d
                    EnP3_AP = d
                    df.EnP1_init.loc[rownum] = EnP1_init
                    df.EnP3_AP.loc[rownum] = EnP3_AP
                    df.Counter1.loc[rownum] = 1
                    df.Counter2.loc[rownum] = 1
                    df.Position.loc[rownum] = 2
                    df.P_L.loc[rowum] = df.close.loc[rownum] - EnP1_init + df.close.loc[rownum] - EnP3_AP
                    RE_CAN = False
                    df.RE_CAN.loc[rownum] = RE_CAN
                    rownum += 1
                else:
                    df.RE_CAN.loc[rownum] = df.RE_CAN.loc[rownum-1] 
                    continue
            
            # case 7:  stop out both positions and iterate until next TrendDay=0 instance        
            elif df.Position.loc[rownum] == 2 and df.low.loc[rownum] < b:
                ExP3_SL = b - tick
                ExP4_SL = b - tick
                df.ExP1_SL.loc[rownum] = ExP1_SL
                df.ExP2_SL.loc[rownum] = ExP2_SL
                #SL = True
                df.P_L.loc[rownum] = (ExP3_SL - EnP1_init) + (ExP4_SL - EnP3_AP)
                rownum += 1
                while df.TrendDay.loc[rownum] != 0:
                    rownum += 1
                break
            break    
        
    else:
        rownum += 1        

     
        
        
    

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
